In [18]:
import os
#os.chdir("../")
%pwd

'd:\\Projects\\Learning\\Wine-Quality-Prediction'

In [19]:
# set MLFLOW_TRACKING_URI=https://dagshub.com/pradeeppvj8/Wine-Quality-Prediction.mlflow
# set MLFLOW_TRACKING_USERNAME=pradeeppvj8
# set MLFLOW_TRACKING_PASSWORD=f831428c26a708547e8a1e4d90cb85cbeb9bdce3

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/pradeeppvj8/Wine-Quality-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "pradeeppvj8"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f831428c26a708547e8a1e4d90cb85cbeb9bdce3"

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [25]:
from wqpproject.constants import *
from wqpproject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, 
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

     

In [26]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from wqpproject.utils.common import save_json
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            scores = {"rmse":rmse,"mae":mae,"r2":r2}

            # Save metrics data into a json file
            save_json(path= Path(self.config.metric_file_name), data= scores)

            # Log data into mlflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            if tracking_url_type_store != "File":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNet")
            else:
                mlflow.sklearn.log_model(model, "model")


In [27]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-09-10 20:34:54,257: INFO: common: yaml file : config\config.yaml loaded successfully]
[2023-09-10 20:34:54,261: INFO: common: yaml file : params.yaml loaded successfully]
[2023-09-10 20:34:54,268: INFO: common: yaml file : schema.yaml loaded successfully]
[2023-09-10 20:34:54,270: INFO: common: Created directory at : artifacts]
[2023-09-10 20:34:54,273: INFO: common: Created directory at : artifacts/model_evaluation]
[2023-09-10 20:34:56,174: INFO: common: Json file is saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticNet' already exists. Creating a new version of this model...
2023/09/10 20:35:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNet, version 2
Created version '2' of model 'ElasticNet'.
